In [2]:

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.cross_validation import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.pipeline import Pipeline
from sklearn.svm import OneClassSVM
import cPickle as pickle

In [3]:
# load data
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')


In [4]:
# remove constant columns
remove = []
for col in df_train.columns:
    if df_train[col].std() == 0:
        remove.append(col)

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)


In [5]:

# remove duplicated columns
remove = []
c = df_train.columns
for i in range(len(c)-1):
    v = df_train[c[i]].values
    for j in range(i+1,len(c)):
        if np.array_equal(v,df_train[c[j]].values):
            remove.append(c[j])

df_train.drop(remove, axis=1, inplace=True)
df_test.drop(remove, axis=1, inplace=True)



In [54]:
y_train = df_train['TARGET'].values
X_train = df_train.drop(['ID','TARGET'], axis=1).values
id_test = df_test['ID']
X_test = df_test.drop(['ID'], axis=1).values
# length of dataset
len_train = len(X_train)
len_test  = len(X_test)

# ImputeFrequent values to -9999999 to most frequent

In [55]:
from sklearn.preprocessing import Imputer

In [56]:
imp = Imputer(missing_values=-999999,strategy="most_frequent",axis=0)

In [57]:
X_train = imp.fit_transform(X_train)

In [58]:
X_test = imp.transform(X_test)

In [59]:
[x for x in X_train if len(set(x))==0]

[]

#Min Max Scale

In [60]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
X_train = min_max_scaler.fit_transform(X_train)
X_test = min_max_scaler.transform(X_test)

#split_in_2_datasets

In [62]:
# 1. Standard_scaling
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
#####################################
X_train_ss = ss.fit_transform(X_train)
X_test_ss = ss.transform(X_test)

In [51]:
# 2. log(1+X) Transformation and SS#
from sklearn.preprocessing import FunctionTransformer
log1p_trans = FunctionTransformer(np.log1p)
X_train_log1p = log1p_trans.transform(X_train)
X_test_log1p = log1p_trans.transform(X_test)
ss = StandardScaler()
#############################################
X_train_log = ss.fit_transform(X_train_log1p)
X_test_log = ss.transform(X_test_log1p)

ImportError: cannot import name FunctionTransformer

# Get a CV SET for stacking

This cv data will be used for tuning 2nd level model

In [64]:
#X_train_ss_stacker, X_cv_ss_stacker, y_train_ss_stacker, y_cv_ss_stacker = train_test_split(X_train_ss, y_train, test_size=0.2, 
# stratify = y_train,random_state=42)
X_train_ss_stacker, X_cv_ss_stacker, y_train_ss_stacker, y_cv_ss_stacker = 
train_test_split(X_train_ss, y_train, test_size=0.2, random_state=42)

In [66]:
#now treat X_train_ss_stacker as training and X_cv_ss_stacker as x_test

In [ ]:
# X_train_ss_stacker will have 4fold cv and will create blend_test also 

In [6]:
from time import time
from operator import itemgetter
from scipy.stats import randint as sp_randint

from sklearn.grid_search import GridSearchCV, RandomizedSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier

In [50]:
# classifier
from scipy.stats import randint as sp_randint
from sklearn.grid_search import GridSearchCV, RandomizedSearchCV

#Best CV Scores
#num_trees:366,max_depth:15,max_features:auto,min_samples_split:2,min_samples_leaf:6,criterion:entropy,mean:0.837152757842,std:0.0087282640641

clfrf = RandomForestClassifier(n_estimators=366,max_depth=15,max_features='auto',min_samples_split=2,
                             min_samples_leaf=6,criterion='entropy')

clfrf.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=15, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=6, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=366, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [52]:
# fitting
# GBT:learning_rate:0.03,n_estimators:785,subsample:0.85,min_samples_split:3,min_samples_leaf:10,min_weight_fraction_leaf:0,max_depth:3,mean:0.837373013071,std:0.00851364001698
from sklearn.ensemble import GradientBoostingClassifier
clfgbt = GradientBoostingClassifier(learning_rate=.03,n_estimators=785,
                                          subsample=0.85,min_samples_split=3,
                                          min_samples_leaf=10,max_depth=3)

clfgbt.fit(X_train, y_train)


GradientBoostingClassifier(init=None, learning_rate=0.03, loss='deviance',
              max_depth=3, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=10, min_samples_split=3,
              min_weight_fraction_leaf=0.0, n_estimators=785,
              random_state=None, subsample=0.85, verbose=0,
              warm_start=False)

In [57]:
#print('Overall AUC:', roc_auc_score(y_train, clf.predict_proba(X_train)[:,1]))

# predicting
y_pred_rf= clfrf.predict_proba(X_test)[:,1]
y_pred_gbt= clfgbt.predict_proba(X_test)[:,1]
xgb_res = pd.read_csv('submission_kagglr.csv')
y_pred_xgb = xgb_res['TARGET']

# cREATING DATA BY ENSEMBLING
submission = pd.DataFrame({"ID":id_test, "RF":y_pred_rf,"XGB":y_pred_xgb,"GBT":y_pred_gbt})
submission['TARGET'] = .60*submission['XGB']+.10*submission['GBT']+.30*submission['RF']
submission = submission[['ID','TARGET']]

submission.to_csv("submission_ENS2.csv", index=False)
print('Completed!')

Completed!


In [54]:
submission.head()

,ID,TARGET
0,2,0.044794
1,5,0.053102
2,6,0.002526
3,7,0.015318
4,9,0.003319


In [93]:
import csv
import random
import numpy as np
from sklearn.cross_validation import StratifiedKFold
from sklearn import metrics
import pandas as pd

class stacker(object):
    def __init__(self , x_test , x_train , y_train,  base_clf_list=[], blender_clf_list = [], 
                 random_seed = 0,n_folds = 5, eval_metric='roc_auc'):
        """ initializes a stacker object """
        self.__base_clf_list = base_clf_list
        self.__blender_clf_list = blender_clf_list
        self.__random_seed     = random_seed
        self.__n_folds = n_folds
        self.__X_test = x_test
        self.__X = x_train
        self.__y = y_train
        # Number of training data x Number of classifiers
        self.__blend_train = np.zeros((self.__X.shape[0], len(self.__base_clf_list))) 
         # Number of testing data x Number of classifiers
        self.__blend_test = np.zeros((self.__X_test.shape[0], len(self.__base_clf_list)))
        self.__eval_metric=eval_metric
        self.__clf_fold_auc = {}
        self.__blender_fold_auc = {}
        self.__skf = list(StratifiedKFold(self.__y, self.__n_folds,shuffle=True,random_state=self.__random_seed))
        self.__blender_preds = np.zeros((self.__X_test.shape[0], len(self.__blender_clf_list)))
        self.__id_test = id_test

    def train_all_base_classifiers(self):
        """ trains the Blender creates a blended_test and blended_train df"""
        skf = self.__skf
         # For each classifier, we train the number of fold times (=len(skf))
        for j, clf in enumerate(self.__base_clf_list):
            print 'Training classifier [%s] [%s]' % (j,clf[1])
            # Number of testing data x Number of folds , we will take the mean of the predictions later
            blend_test_j = np.zeros((self.__X_test.shape[0], len(skf))) 
            for i, (train_index, cv_index) in enumerate(skf):
                print 'Fold [%s]' % (i)
                # This is the training and validation set
                X_tr = self.__X[train_index]
                Y_tr = self.__y[train_index]
                X_cv = self.__X[cv_index]
                Y_cv = self.__y[cv_index]
                clf[0].fit(X_tr, Y_tr)
                class_preds = clf[0].predict_proba(X_cv)[:,1]
                eval_metric = self.__eval_metric
                auc_score = roc_auc_score(Y_cv, class_preds)
                print "auc_score for fold:",auc_score
                if clf[1] in self.__clf_fold_auc:
                    self.__clf_fold_auc[clf[1]].append(auc_score)
                else:
                    self.__clf_fold_auc[clf[1]] = [auc_score]
                # This output will be the basis for our blended classifier to train against,
                # which is also the output of our classifiers
                self.__blend_train[cv_index, j] = class_preds
                blend_test_j[:, i] = clf[0].predict_proba(self.__X_test)[:,1]
            # Take the mean of the predictions of the cross validation set
            print "cv_score_mean:",np.mean(self.__clf_fold_auc[clf[1]]),"and cv_score_std:",np.std(self.__clf_fold_auc[clf[1]])
            self.__blend_test[:, j] = blend_test_j.mean(1)

    def add_base_classifer(self,clf):
        self.__base_clf_list.append(clf)
        # add a new column to both blended train and test
        self.__blend_train = np.c_[self.__blend_train,np.zeros(self.__X.shape[0])]
        self.__blend_test = np.c_[self.__blend_test,np.zeros(self.__X_test.shape[0])]
        print 'Training classifier [%s] [%s]' % (len(self.__base_clf_list),clf[1])
        # Number of testing data x Number of folds , we will take the mean of the predictions later
        blend_test_j = np.zeros((self.__X_test.shape[0], len(self.__skf))) 
        for i, (train_index, cv_index) in enumerate(self.__skf):
            print 'Fold [%s]' % (i)
            # This is the training and validation set
            X_tr = self.__X[train_index]
            Y_tr = self.__y[train_index]
            X_cv = self.__X[cv_index]
            Y_cv = self.__y[cv_index]
            clf[0].fit(X_tr, Y_tr)
            class_preds = clf[0].predict_proba(X_cv)[:,1]
            eval_metric = self.__eval_metric
            auc_score = roc_auc_score(Y_cv, class_preds)
            print "auc_score for fold:",auc_score
            if clf[1] in self.__clf_fold_auc:
                self.__clf_fold_auc[clf[1]].append(auc_score)
            else:
                self.__clf_fold_auc[clf[1]] = [auc_score]
            # This output will be the basis for our blended classifier to train against,
            # which is also the output of our classifiers
            self.__blend_train[cv_index, len(self.__base_clf_list)-1] = class_preds
            blend_test_j[:, i] = clf[0].predict_proba(self.__X_test)[:,1]
        # Take the mean of the predictions of the cross validation set
        print "cv_score_mean:",np.mean(self.__clf_fold_auc[clf[1]]),"and cv_score_std:",np.std(self.__clf_fold_auc[clf[1]])
        self.__blend_test[:, len(self.__base_clf_list)-1] = blend_test_j.mean(1)
    
    def add_blenders(self,blender):
        self.__blender_clf_list.append(blender)

    def remove_blenders(self,blender):
        self.__blender_clf_list.remove(blender)
    
    def find_cv_scores_all_blenders(self):
        for blender,blendername in self.__blender_clf_list:
            print "blender_Name:",blendername,":"
            for i, (train_index, cv_index) in enumerate(self.__skf):
                X_tr = self.__blend_train[train_index]
                Y_tr = self.__y[train_index]
                X_cv = self.__blend_train[cv_index]
                Y_cv = self.__y[cv_index]
                blender.fit(X_tr, Y_tr)
                class_preds = blender.predict_proba(X_cv)[:,1]
                auc_score = roc_auc_score(Y_cv, class_preds)
                print "Fold",i+1,"CV Score:",auc_score
                if blendername in self.__blender_fold_auc:
                    self.__blender_fold_auc[blendername].append(auc_score)
                else:
                    self.__blender_fold_auc[blendername] = [auc_score]
            print "cv_score_mean:",np.mean(self.__blender_fold_auc[blendername]),"and cv_score_std:",np.std(self.__blender_fold_auc[blendername])
        
    def train_all_blenders(self):
        self.__blender_preds = np.zeros((self.__X_test.shape[0], len(self.__blender_clf_list)))
        for i,blender_obj in enumerate(self.__blender_clf_list):
            blender,blender_name = blender_obj
            print "Training Blender #",i+1,"|",blender_name
            blender.fit(self.__blend_train,self.__y)
            blender_probs = blender.predict_proba(self.__blend_test)[:,1]
            self.__blender_preds[:, i] = blender_probs

    def get_weighted_blender_submission(self,submission_name,req_blender_list=[], blender_weights=0):
        """blender_weights should be a np.array"""
        if len(req_blender_list)==0:
            req_blender_list = [x[1] for x in self.__blender_clf_list]
        blender_dataframe = pd.DataFrame(self.__blender_preds,columns= [x[1] for x in self.__blender_clf_list])
        if blender_weights==0:
            #print blender_dataframe
            #print req_blender_list
            blender_df = blender_dataframe[req_blender_list]
            
            new_blender_array = np.sum(np.array(blender_df),axis=1)/len(req_blender_list)
        else:
            blender_weights = np.array(blender_weights)
            blender_df = blender_dataframe[req_blender_list]
            new_blender_array =np.dot(np.array(blender_df),blender_weights)
        submission = pd.DataFrame({"ID":self.__id_test, "TARGET":new_blender_array})
        submission.to_csv(submission_name, index=False)
        return submission

    ### ADD YOU OWN GETTER SETTER FUNCS.
    def get_blend_train(self):
        return self.__blend_train

    def get_blend_test(self):
        return self.__blend_test

    def get_blended_preds_df(self):
        return pd.DataFrame(self.__blender_preds,columns= [x[1] for x in self.__blender_clf_list])

    def __str__(self):
        res = "init:\n" + "n_folds: " + str(self.__n_folds) +" random_seed:"+str(self.__random_seed)+"\n"
        res += "base classifiers:\n"
        for clf_num,clf in enumerate(self.__base_clf_list):
            res += str(clf_num+1)+". "+str(clf) + " \n"
            for i, auc_score in enumerate(self.__clf_fold_auc[clf[1]]):
                res += "fold_"+str(i+1)+"_auc:"+str(auc_score)+"\n"

        res += "Blender classifiers:\n"
        for clf_num,clf in enumerate(self.__blender_clf_list):
            res += str(clf_num+1)+". "+str(clf) + " \n"
            for i, auc_score in enumerate(self.__blender_fold_auc[clf[1]]):
                res += "fold_"+str(i+1)+"_auc:"+str(auc_score)+"\n"
        
        return res


In [94]:
stack2 = stacker(x_test = X_cv_ss_stacker,x_train = X_train_ss_stacker,y_train =y_train_ss_stacker)
#X_train_ss_stacker, X_cv_ss_stacker, y_train_ss_stacker, y_cv_ss_stacker

In [95]:
stack2.add_base_classifer([RandomForestClassifier(n_estimators = 10, criterion = 'gini'),"RF_GINI"])

Training classifier [1] [RF_GINI]
Fold [0]
auc_score for fold: 0.685457288813
Fold [1]
auc_score for fold: 0.674064702274
Fold [2]
auc_score for fold: 0.66895756655
Fold [3]
auc_score for fold: 0.673893067563
Fold [4]
auc_score for fold: 0.697194017661
cv_score_mean: 0.679913328572 and cv_score_std: 0.0101964347124


In [96]:
stack2.add_base_classifer([RandomForestClassifier(n_estimators = 10, criterion = 'entropy'),"RF_ENTROPY"])

Training classifier [2] [RF_ENTROPY]
Fold [0]
auc_score for fold: 0.690217835215
Fold [1]
auc_score for fold: 0.674254704129
Fold [2]
auc_score for fold: 0.665954806129
Fold [3]
auc_score for fold: 0.690976650546
Fold [4]
auc_score for fold: 0.694794876459
cv_score_mean: 0.683239774495 and cv_score_std: 0.0111496599478


In [101]:
from sklearn.ensemble import GradientBoostingClassifier
stack2.add_base_classifer([GradientBoostingClassifier(n_estimators=10),"GB"])

Training classifier [3] [GB]
Fold [0]
auc_score for fold: 0.81978621673
Fold [1]
auc_score for fold: 0.800097542013
Fold [2]
auc_score for fold: 0.804099974322
Fold [3]
auc_score for fold: 0.823448243887
Fold [4]
auc_score for fold: 0.829046834432
cv_score_mean: 0.815295762277 and cv_score_std: 0.0112432263177


In [102]:
x_cv = stack2.get_blend_test()
x_tr = stack2.get_blend_train()

In [118]:
from sklearn.linear_model import LogisticRegression
blender = LogisticRegression()
blender.fit(x_tr,y_train_ss_stacker)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr',
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0)

In [122]:
print "auc:train",roc_auc_score(y_train_ss_stacker,blender.predict_proba(x_tr)[:,1])
print "local_cv_error:test",roc_auc_score(y_cv_ss_stacker,blender.predict_proba(x_cv)[:,1])

auc:train 0.818490102433
local_cv_error:test 0.824884917451


In [87]:
blender.predict(x_cv)

IndexError: too many indices for array

In [187]:
stack3 = stacker(x_test = X_test,x_train = X_train,y_train =y_train,id_test=id_test, base_clf_list=[[RandomForestClassifier(n_estimators = 2, criterion = 'entropy'),"RF_ENTROPY"],
                                                                                   [RandomForestClassifier(n_estimators = 2, criterion = 'gini'),"RF_GINI"]
                                                                                   ]
                ) 

In [167]:
stack3.train_all_base_classifiers()

Training classifier [0] [RF_ENTROPY]
Fold [0]
auc_score for fold: 0.587837444315
Fold [1]
auc_score for fold: 0.586507960522
Fold [2]
auc_score for fold: 0.585919316109
Fold [3]
auc_score for fold: 0.597974521303
Fold [4]
auc_score for fold: 0.58788051508
cv_score_mean: 0.589223951466 and cv_score_std: 0.00444065124784
Training classifier [1] [RF_GINI]
Fold [0]
auc_score for fold: 0.595458187607
Fold [1]
auc_score for fold: 0.597067218473
Fold [2]
auc_score for fold: 0.58707318799
Fold [3]
auc_score for fold: 0.60299280909
Fold [4]
auc_score for fold: 0.593728641553
cv_score_mean: 0.595264008943 and cv_score_std: 0.00514724241871


In [255]:
stack4 = stacker(x_test = X_test,x_train = X_train,y_train =y_train,id_test=id_test, base_clf_list=[[RandomForestClassifier(n_estimators = 2, criterion = 'entropy'),"RF_ENTROPY"],
                                                                                   [RandomForestClassifier(n_estimators = 2, criterion = 'gini'),"RF_GINI"]
                                                                                   ],
                 
                 blender_clf_list = [[RandomForestClassifier(n_estimators = 2, criterion = 'entropy'),"BLEND_RF_ENTROPY"],
                                                                                   [RandomForestClassifier(n_estimators = 2, criterion = 'gini'),"BLEND_RF_GINI"]
                                                                                   ]
                ) 

In [256]:
stack4.train_all_base_classifiers()

Training classifier [0] [RF_ENTROPY]
Fold [0]
auc_score for fold: 0.601556465779
Fold [1]
auc_score for fold: 0.604232041248
Fold [2]
auc_score for fold: 0.584394983439
Fold [3]
auc_score for fold: 0.603993230476
Fold [4]
auc_score for fold: 0.59159561713
cv_score_mean: 0.597154467614 and cv_score_std: 0.00787329116686
Training classifier [1] [RF_GINI]
Fold [0]
auc_score for fold: 0.590317422147
Fold [1]
auc_score for fold: 0.604126251307
Fold [2]
auc_score for fold: 0.598296676694
Fold [3]
auc_score for fold: 0.593156671037
Fold [4]
auc_score for fold: 0.579752596971
cv_score_mean: 0.593129923631 and cv_score_std: 0.00817897680863


In [257]:
stack4.train_all_blenders()

Training Blender # 1 | BLEND_RF_ENTROPY
Training Blender # 2 | BLEND_RF_GINI


In [258]:
stack4.find_cv_scores_all_blenders()

blender_Name: BLEND_RF_ENTROPY :
Fold 1 CV Score: 0.602361151841
Fold 2 CV Score: 0.625628341057
Fold 3 CV Score: 0.596159573553
Fold 4 CV Score: 0.601513229218
Fold 5 CV Score: 0.596663814886
cv_score_mean: 0.604465222111 and cv_score_std: 0.0108707380607
blender_Name: BLEND_RF_GINI :
Fold 1 CV Score: 0.597490833245
Fold 2 CV Score: 0.614910113814
Fold 3 CV Score: 0.595923008772
Fold 4 CV Score: 0.604254460162
Fold 5 CV Score: 0.591763066213
cv_score_mean: 0.600868296441 and cv_score_std: 0.00809205888706


In [259]:
print(stack4)

init:
n_folds: 5 random_seed:0
base classifiers:
1. [RandomForestClassifier(bootstrap=True, class_weight=None, criterion='entropy',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False), 'RF_ENTROPY'] 
fold_1_auc:0.601556465779
fold_2_auc:0.604232041248
fold_3_auc:0.584394983439
fold_4_auc:0.603993230476
fold_5_auc:0.59159561713
2. [RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=2, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False), 'RF_GINI'] 
fold_1_auc:0.590317422147
fold_2_auc:0.604126251307
fold_3_auc:0.59829

In [238]:
stack4.get_weighted_blender_submission(submission_name= "unw_ble.csv")

       BLEND_RF_ENTROPY  BLEND_RF_GINI
0              0.000000       0.000000
1              0.028725       0.028071
2              0.028725       0.028071
3              0.028725       0.028071
4              0.028725       0.028071
5              0.000000       0.000000
6              0.000000       0.000000
7              0.028725       0.028071
8              0.000000       0.000000
9              0.000000       0.000000
10             0.028725       0.028071
11             0.028725       0.028071
12             0.028725       0.028071
13             0.028725       0.028071
14             0.028725       0.028071
15             0.000000       0.000000
16             0.000000       0.000000
17             0.028725       0.028071
18             0.028725       0.028071
19             0.000000       0.000000
20             0.028725       0.028071
21             0.166667       0.000000
22             0.028725       0.028071
23             0.028725       0.028071
24             0.000000  

,ID,TARGET
0,2,0.000000
1,5,0.028398
2,6,0.028398
3,7,0.028398
4,9,0.028398
5,11,0.000000
6,12,0.000000
7,15,0.028398
8,16,0.000000
9,17,0.000000


In [93]:
a.shape[1]-1

3L

In [6]:
train_d = pickle.load(open("BASE_KNN4blend_train.pkl","rb"))
test_d = pickle.load(open("BASE_KNN4blend_test.pkl","rb"))

In [24]:
from sklearn.ensemble import GradientBoostingClassifier
clfgbt = GradientBoostingClassifier(learning_rate=0.001,n_estimators=4650,
                                          subsample=0.8,min_samples_split=9,
                                          min_samples_leaf=9,max_depth=2)

#predsGBT:learning_rate:0.005,n_estimators:551,subsample:0.95,min_samples_split:10,min_samples_leaf:4,min_weight_fraction_leaf:0,max_depth:3,mean:0.844772435022,std:0.00374481377244
#preds2GBT:learning_rate:0.001,n_estimators:4650,subsample:0.8,min_samples_split:8,min_samples_leaf:9,min_weight_fraction_leaf:0,max_depth:2,mean:0.845286774382,std:0.00311551434064

clfgbt.fit(train_d,y_train)

GradientBoostingClassifier(init=None, learning_rate=0.001, loss='deviance',
              max_depth=2, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=9, min_samples_split=9,
              min_weight_fraction_leaf=0.0, n_estimators=4650,
              random_state=None, subsample=0.8, verbose=0,
              warm_start=False)

In [25]:
preds2 = clfgbt.predict_proba(test_d)[:,1]
preds3 = (.3*preds+.7*preds2)

In [26]:

submission = pd.DataFrame({"ID":id_test, "TARGET":preds3})
submission.to_csv("gbt_enstrain.csv", index=False)

In [27]:
preds2

array([ 0.03228861,  0.04665701,  0.0064141 , ...,  0.0064141 ,
        0.0478516 ,  0.0064141 ])

In [28]:
preds3

array([ 0.0322188 ,  0.04812189,  0.00655761, ...,  0.00655761,
        0.04806651,  0.00655761])

In [29]:
preds

array([ 0.03205592,  0.05153992,  0.00689245, ...,  0.00689245,
        0.04856795,  0.00689245])

In [9]:
from sklearn.cross_validation import cross_val_score

cross_val_score(clfgbt, train_d, y_train, scoring='roc_auc', cv=5, n_jobs=1, verbose=1)

[Parallel(n_jobs=1)]: Done   1 jobs       | elapsed:    1.0s
[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    5.4s finished


array([ 0.8407713 ,  0.8352808 ,  0.84565926,  0.85855088,  0.84116289])

In [10]:
a = np.array([ 0.8407713 ,  0.8352808 ,  0.84565926,  0.85855088,  0.84116289])

In [12]:
np.mean(a), np.std(a)

(0.84428502599999999, 0.0078558665836522424)

In [13]:
from sklearn.ensemble import GradientBoostingClassifier
clfgbt = GradientBoostingClassifier(learning_rate=.1,n_estimators=100,
                                          subsample=0.85,min_samples_split=3,
                                          min_samples_leaf=10,max_depth=3)


In [ ]:
a = cross_val_score(clfgbt, train_d, y_train, scoring='roc_auc', cv=5, n_jobs=2, verbose=1)
np.mean(a), np.std(a)

In [9]:
df_train_blend=np.c_[train_d,y_train]

In [12]:
df_train_blend=pd.DataFrame(df_train_blend,columns=['rf','gbt','xgb','rf_bal','nn_ada','nn_sgd','knn_2','knn_4','target'])

In [13]:
df_train_blend.to_csv("train_blend.csv")

In [30]:
clfgbt.feature_importances_

array([ 0.15141854,  0.08890051,  0.26170776,  0.07933218,  0.39268729,
        0.02268647,  0.00271389,  0.00055338])

In [39]:
import numpy as np
from patsy import dmatrices, dmatrix, demo_data

In [50]:
df_train_blend['target'] = df_train_blend['target'].astype('int64')

In [51]:
df_train_blend.dtypes

rf        float64
gbt       float64
xgb       float64
rf_bal    float64
nn_ada    float64
nn_sgd    float64
knn_2     float64
knn_4     float64
target      int64
dtype: object

In [ ]:
!pip install patsy --upgrade

In [52]:
dmatrices("target ~ rf*gbt", df_train_blend)

ValueError: For numerical factors, num_columns must be an int